**INTRODUÇÃO**
Identificar e satisfazer curiosidades sobre fatores que ocasionam o churn de alguns clientes.

In [ ]:
# manipulação dos dados
import numpy as np
import pandas as pd

# sistema
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# exploração dos dados
import matplotlib.pyplot as plt
import seaborn as sns

# configurações
plt.style.use('ggplot')

In [ ]:
churn_data = pd.read_csv('/kaggle/input/bank-customer-churn-dataset/Bank Customer Churn Prediction.csv')
churn_data.head()

In [ ]:
churn_data.nunique().sort_values()

In [ ]:
churn_data.country.unique()

In [ ]:
min_balance = churn_data.balance.min()
max_balance = churn_data.balance.max()
f'Saldo mínimo: {min_balance} e Saldo máximo: {max_balance}'

In [ ]:
min_salary = churn_data.estimated_salary.min()
max_salary = churn_data.estimated_salary.max()
f'Salário mínimo: {min_salary} e Salário máximo: {max_salary}'

In [ ]:
zero_balance = churn_data.query("balance == 0")

In [ ]:
churn_data.query(f"balance in {churn_data.balance.nlargest(3).to_list()} or estimated_salary in {churn_data.estimated_salary.nlargest(3).to_list()}") 

ANÁLISE EXPLORATÓRIA

In [ ]:
gender_count = churn_data.gender.value_counts(normalize=True)
gender_count

In [ ]:
gender_count.plot.bar()
plt.title('Distribuição de Gênero')
_ = plt.text(-0.1, 0.55, gender_count[0])
_ = plt.text(0.91, 0.46, gender_count[1])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 10))
_ = sns.countplot(data=churn_data, x='country', ax=axes[0][0])
_ = sns.countplot(data=churn_data, x='products_number', ax=axes[0][1])
_ = sns.countplot(data=churn_data, x='credit_card', ax=axes[1][0])
_ = sns.countplot(data=churn_data, x='active_member', ax=axes[1][1])
_ = plt.tight_layout()

In [ ]:
_ = plt.hist(churn_data.balance, bins=50, color='purple')

In [ ]:
print(f'O número de clientes que não possuem saldo é de {len(zero_balance)}.')

In [ ]:
_ = sns.relplot(x = 'estimated_salary', y = 'balance', data=churn_data, col='country', color='purple')

In [ ]:
_ = sns.jointplot(x='estimated_salary', y='age', data=churn_data)

ANÁLISE DE CHURN

In [ ]:
_ = churn_data.churn.value_counts(1).plot.bar()
_ = plt.title('Distribuição de Churn')
_ = plt.text(-0.06, 0.815, churn_data.churn.value_counts(0)[0])
_ = plt.text(0.93, 0.22, churn_data.churn.value_counts(0)[1])
_ = plt.ylim(0, 0.88)

In [ ]:
fig, axes = plt.subplots(2, 1)
_ = sns.boxplot(data=churn_data, x = 'churn', y='estimated_salary', ax=axes[0])
_ = sns.boxplot(data=churn_data, x = 'churn', y='balance', ax=axes[1])
_ = fig.tight_layout()

In [ ]:
churn_data.describe()

In [ ]:
churn_data.groupby('churn')['balance'].median()

In [ ]:
churn_data.groupby('churn')['balance'].mean()

In [ ]:
churn_data.groupby('churn').agg({'balance': ['mean', 'median']})

In [ ]:
agg_func = {
    'estimated_salary':['min', 'median', 'mean', 'std', skew],
    'balance':['min', 'median', 'mean', 'std', skew],
    'age':['min', 'median', 'mean', 'std', skew]
}

churn_data.groupby('churn').agg(agg_func)

In [ ]:
sns.boxplot(data=churn_data, x = 'churn', y='age')

In [ ]:
churn_data.groupby('churn').agg({'credit_score':['min','max','mean', 'median']})

HIPÓTESE

In [ ]:
france_data = churn_data[churn_data['country'] == 'France']
spain_data = churn_data[churn_data['country'] == 'Spain']
germany_data = churn_data[churn_data['country'] == 'Germany']

In [ ]:
sns.countplot(data=churn_data, x='country', hue='churn')
# France
_ = plt.text(-0.3, 4250, str(france_data['churn'].value_counts(0)[0]))
_ = plt.text(0.12, 850, str(france_data['churn'].value_counts(0)[1]))
# Spain
_ = plt.text(0.70, 2100, str(spain_data['churn'].value_counts(0)[0]))
_ = plt.text(1.12, 450, str(spain_data['churn'].value_counts(0)[1]))
# Germany
_ = plt.text(1.7, 1740, str(germany_data['churn'].value_counts(0)[0]))
_ = plt.text(2.12, 850, str(germany_data['churn'].value_counts(0)[1]))

In [ ]:
churn_data.groupby('country')['churn'].mean().sort_values()

In [ ]:
churn_data.groupby('credit_card')['churn'].mean().sort_values()

In [ ]:
churn_data.groupby('products_number')['churn'].mean().sort_values()

In [ ]:
products_churn = sns.countplot(data=churn_data, x='products_number', hue='churn')
sns.move_legend(products_churn, loc='upper right')

In [ ]:
churn_data.groupby('gender')['churn'].mean().sort_values()

CONCLUSÃO

A análise acima mostra alguns fatores relevantes para o churn, como o saldo, idade, gênero, país e número de produtos.
Nota-se que clientes com maior saldo tem mais propensão ao churn.
Clientes com idade média acima de 45 anos, mulheres, alemães e aqueles que possuem três ou mais produtos também tem um índice de churn maior.